1. [Importing Packages](#1)
1. [Importing Dataset](#2)
1. [Data Understanding](#3)
1. [Feature Engineering](#4)
  1. [Changing 'default'](#4.1)
  1. [Changing 'housing'](#4.2)
  1. [Changing 'loan'](#4.3)
  1. [Changing 'y'](#4.4)
1. [Encoding](#5)
  1. [job](#5.1)
  1. [marital](#5.2)
  1. [education](#5.3)
  1. [contact](#5.4)
  1. [day](#5.5)
  1. [month](#5.6)
1. [Feature Scaling](#6)
1. [Train-Test Split](#7)
1. [Model & Predict](#8)
1. [Model Tuning](#9)
1. [Final Model](#10)

<a id="1"></a> <br>
# 1- Kütüphanelerin Tanımlanması

In [171]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split

import warnings
warnings.filterwarnings('ignore')

<a id="2"></a> <br>
# 2- Veriyi Okuma

In [172]:
df = pd.read_csv('Data/term-deposit-marketing-2020.csv')

In [173]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1,no


<a id="3"></a> <br>
# 3- Veriyi Anlama

In [174]:
# Verilerdeki kolonlarin veri tiplerine bakiyoruz

print('---------------------------------')
print(df.info())
print('---------------------------------')

---------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        40000 non-null  int64 
 1   job        40000 non-null  object
 2   marital    40000 non-null  object
 3   education  40000 non-null  object
 4   default    40000 non-null  object
 5   balance    40000 non-null  int64 
 6   housing    40000 non-null  object
 7   loan       40000 non-null  object
 8   contact    40000 non-null  object
 9   day        40000 non-null  int64 
 10  month      40000 non-null  object
 11  duration   40000 non-null  int64 
 12  campaign   40000 non-null  int64 
 13  y          40000 non-null  object
dtypes: int64(5), object(9)
memory usage: 4.3+ MB
None
---------------------------------


In [175]:
# Verilerdeki bos satirlara bakiyoruz

print('---------------------------------')
print(df.isna().sum())
print('---------------------------------')

---------------------------------
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
y            0
dtype: int64
---------------------------------


In [176]:
# Verilerdeki kolonlarin unique eleman sayilarina bakiyoruz

print('---------------------------------')
for i in df.columns:
    print(i,' kolonundaki unique eleman sayisi',df[i].nunique())
print('---------------------------------')

---------------------------------
age  kolonundaki unique eleman sayisi 70
job  kolonundaki unique eleman sayisi 12
marital  kolonundaki unique eleman sayisi 3
education  kolonundaki unique eleman sayisi 4
default  kolonundaki unique eleman sayisi 2
balance  kolonundaki unique eleman sayisi 6849
housing  kolonundaki unique eleman sayisi 2
loan  kolonundaki unique eleman sayisi 2
contact  kolonundaki unique eleman sayisi 3
day  kolonundaki unique eleman sayisi 31
month  kolonundaki unique eleman sayisi 11
duration  kolonundaki unique eleman sayisi 1535
campaign  kolonundaki unique eleman sayisi 48
y  kolonundaki unique eleman sayisi 2
---------------------------------


In [177]:
df.describe()

,age,balance,day,duration,campaign
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,40.544600,1274.277550,16.017225,254.824300,2.882175
std,9.641776,2903.769716,8.278127,259.366498,3.239051
min,19.000000,-8019.000000,1.000000,0.000000,1.000000
25%,33.000000,54.000000,8.000000,100.000000,1.000000
50%,39.000000,407.000000,17.000000,175.000000,2.000000
75%,48.000000,1319.000000,21.000000,313.000000,3.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000


<a id="4"></a> <br>
# 4- Öznitelik Çıkarımı (Feature Engineering)

<a id="4.1"></a> <br>
## 'default' Değişimi

In [178]:
# 'default' kolonundaki degerleri 'yes' ise 1 'no' ise 0 olarak degistiriyoruz

df['default'] = df['default'].apply(lambda x: 1 if x =='yes' else 0)

In [179]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,0,2143,yes,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,0,29,yes,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,0,2,yes,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,0,1506,yes,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,0,1,no,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,0,395,no,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,0,3340,no,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,0,200,no,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,0,1047,no,no,cellular,3,jun,342,1,no


<a id="4.2"></a> <br>
## 'housing' Değişimi

In [180]:
# 'housing' kolonundaki degerleri 'yes' ise 1 'no' ise 0 olarak degistiriyoruz

df['housing'] = df['housing'].apply(lambda x: 1 if x =='yes' else 0)

In [181]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,0,2143,1,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,0,29,1,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,0,2,1,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,0,1506,1,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,0,1,0,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,0,395,0,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,0,3340,0,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,0,200,0,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,0,1047,0,no,cellular,3,jun,342,1,no


<a id="4.3"></a> <br>
## 'loan' Değişimi

In [182]:
# 'loan' kolonundaki degerleri 'yes' ise 1 'no' ise 0 olarak degistiriyoruz

df['loan'] = df['loan'].apply(lambda x: 1 if x =='yes' else 0)

In [183]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,no
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,no
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,0,395,0,0,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,0,3340,0,0,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,0,200,0,0,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,0,1047,0,0,cellular,3,jun,342,1,no


<a id="4.4"></a> <br>
## 'y' Değişimi

In [184]:
# 'y' kolonundaki degerleri 'yes' ise 1 'no' ise 0 olarak degistiriyoruz

df['y'] = df['y'].apply(lambda x: 1 if x =='yes' else 0)

In [185]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,0,395,0,0,cellular,3,jun,107,1,0
39996,30,management,single,tertiary,0,3340,0,0,cellular,3,jun,238,3,1
39997,54,admin,divorced,secondary,0,200,0,0,cellular,3,jun,170,1,1
39998,34,management,married,tertiary,0,1047,0,0,cellular,3,jun,342,1,0


<a id="5"></a> <br>
# 5- Encoding

In [186]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,0
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,0
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,0
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,0
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,0,395,0,0,cellular,3,jun,107,1,0
39996,30,management,single,tertiary,0,3340,0,0,cellular,3,jun,238,3,1
39997,54,admin,divorced,secondary,0,200,0,0,cellular,3,jun,170,1,1
39998,34,management,married,tertiary,0,1047,0,0,cellular,3,jun,342,1,0


In [187]:
encod_cols = ['job','marital','education','contact','day','month']

<a id="5.1"></a> <br>
## job

In [188]:
print('---------------------------------')
print('Dummy öncesi shape: {}'.format(df.shape))
df = pd.get_dummies(df, columns=[encod_cols[0]], drop_first=True)
print('Dummy sonrası shape: {}'.format(df.shape))
print('---------------------------------')

---------------------------------
Dummy öncesi shape: (40000, 14)
Dummy sonrası shape: (40000, 24)
---------------------------------


<a id="5.2"></a> <br>
## marital

In [189]:
print('---------------------------------')
print('Dummy öncesi shape: {}'.format(df.shape))
df = pd.get_dummies(df, columns=[encod_cols[1]], drop_first=True)
print('Dummy sonrası shape: {}'.format(df.shape))
print('---------------------------------')

---------------------------------
Dummy öncesi shape: (40000, 24)
Dummy sonrası shape: (40000, 25)
---------------------------------


<a id="5.3"></a> <br>
## education

In [190]:
print('---------------------------------')
print('Dummy öncesi shape: {}'.format(df.shape))
df = pd.get_dummies(df, columns=[encod_cols[2]], drop_first=True)
print('Dummy sonrası shape: {}'.format(df.shape))
print('---------------------------------')

---------------------------------
Dummy öncesi shape: (40000, 25)
Dummy sonrası shape: (40000, 27)
---------------------------------


<a id="5.4"></a> <br>
## contact

In [191]:
print('---------------------------------')
print('Dummy öncesi shape: {}'.format(df.shape))
df = pd.get_dummies(df, columns=[encod_cols[3]], drop_first=True)
print('Dummy sonrası shape: {}'.format(df.shape))
print('---------------------------------')

---------------------------------
Dummy öncesi shape: (40000, 27)
Dummy sonrası shape: (40000, 28)
---------------------------------


<a id="5.5"></a> <br>
## day

In [192]:
print('---------------------------------')
print('Dummy öncesi shape: {}'.format(df.shape))
df = pd.get_dummies(df, columns=[encod_cols[4]], drop_first=True)
print('Dummy sonrası shape: {}'.format(df.shape))
print('---------------------------------')

---------------------------------
Dummy öncesi shape: (40000, 28)
Dummy sonrası shape: (40000, 57)
---------------------------------


<a id="5.6"></a> <br>
## month

In [193]:
print('---------------------------------')
print('Dummy öncesi shape: {}'.format(df.shape))
df = pd.get_dummies(df, columns=[encod_cols[5]], drop_first=True)
print('Dummy sonrası shape: {}'.format(df.shape))
print('---------------------------------')

---------------------------------
Dummy öncesi shape: (40000, 57)
Dummy sonrası shape: (40000, 66)
---------------------------------


In [194]:
df

,age,default,balance,housing,loan,duration,campaign,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct
0,58,0,2143,1,0,261,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,44,0,29,1,0,151,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,33,0,2,1,1,76,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,47,0,1506,1,0,92,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,33,0,1,0,0,198,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,0,395,0,0,107,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
39996,30,0,3340,0,0,238,3,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
39997,54,0,200,0,0,170,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
39998,34,0,1047,0,0,342,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


<a id="6"></a> <br>
# 6- Feature Scaling

In [195]:
x = df.drop('y',axis=1)
y = df[['y']]

In [196]:
scalerx = MinMaxScaler().fit(x)
x = pd.DataFrame(scalerx.transform(x),columns=x.columns)

<a id="7"></a> <br>
# 7- Train-Test Split

In [197]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

<a id="8"></a> <br>
# 8- Model & Predict

In [198]:
xgb = XGBClassifier(n_jobs=-1,verbosity=2)

In [199]:
xgb.fit(x_train,y_train)

[12:19:44] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 0 pruned nodes, max_depth=6
[12:19:44] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 112 extra nodes, 0 pruned nodes, max_depth=6
[12:19:44] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 112 extra nodes, 0 pruned nodes, max_depth=6
[12:19:44] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 0 pruned nodes, max_depth=6
[12:19:44] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 0 pruned nodes, max_depth=6
[12:19:44] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 98 extr

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=2)

### Test Accuracy

In [200]:
accuracy_score(y_test, xgb.predict(x_test))

0.934

### Test+Train Accuracy

In [201]:
accuracy_score(y, xgb.predict(x))

0.96585

### Test Confusion Matrix

In [202]:
confusion_matrix(y_test, xgb.predict(x_test))

array([[5448,  127],
       [ 269,  156]])

### Test+Train Confusion Matrix

In [203]:
confusion_matrix(y, xgb.predict(x))

array([[36796,   308],
       [ 1058,  1838]])

<a id="9"></a> <br>
# 9- Model Tuning

In [204]:
xgb = XGBClassifier(n_jobs=-1,verbosity=2)

In [205]:
xgb_params = {"learning_rate": [0.1,0.3,0.5],
             "max_depth": [4,6,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.5,1]}

In [206]:
xgb_cv_model  = GridSearchCV(xgb,xgb_params, cv = 5, n_jobs = -1, verbose = 2).fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 16.8min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 63.7min finished


[13:27:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=4
[13:27:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=4
[13:27:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=4
[13:27:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=4
[13:27:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=4
[13:27:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nod

In [209]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 500}

In [218]:
xgb_tuned = XGBClassifier(colsample_bytree=0.5,learning_rate=0.1,max_depth=4,n_estimators=500,n_jobs=-1)
xgb_tuned.fit(x_train,y_train)

[14:07:39] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=4
[14:07:39] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=4
[14:07:39] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=4
[14:07:40] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 28 extra nodes, 0 pruned nodes, max_depth=4
[14:07:40] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 30 extra nodes, 0 pruned nodes, max_depth=4
[14:07:40] INFO: /Users/runner/miniforge3/conda-bld/xgboost_1598185652448/work/src/tree/updater_prune.cc:101: tree pruning end, 26 extra nod

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Test Accuracy

In [214]:
accuracy_score(y_test, xgb_tuned.predict(x_test))

0.9368333333333333

### Test+Train Accuracy

In [215]:
accuracy_score(y, xgb_tuned.predict(x))

0.95515

### Test Confusion Matrix

In [216]:
confusion_matrix(y_test, xgb_tuned.predict(x_test))

array([[5468,  107],
       [ 272,  153]])

### Test+Train Confusion Matrix

In [217]:
confusion_matrix(y, xgb_tuned.predict(x))

array([[36684,   420],
       [ 1374,  1522]])

<a id="10"></a> <br>
# 10- Final Model